In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#path="/content/drive/MyDrive/Project 1/COVID.xlsx"
#df=pd.read_excel(path)

In [3]:
# Load the Excel file
file_path = '/content/drive/MyDrive/Project 1/Canada_Hosp1_COVID_InpatientData.xlsx'
xls = pd.ExcelFile(file_path)

# Check the sheet names
xls.sheet_names

['Data-at-admission',
 'Days-breakdown',
 'Hospital-length-of-stay',
 'Medication-Static-List']

In [ ]:
# Load each relevant sheet into a DataFrame
data_at_admission = pd.read_excel(xls, 'Data-at-admission')
days_breakdown = pd.read_excel(xls, 'Days-breakdown')
hospital_length_of_stay = pd.read_excel(xls, 'Hospital-length-of-stay')

# Display the first few rows of each DataFrame to understand the structure
data_at_admission.head(), days_breakdown.head(), hospital_length_of_stay.head()

In [5]:
features = ['age', 'sex', 'height', 'weight', 'comorbidities',
            'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate', 'respiratory_rate',
            'oxygen_saturation', 'temperature', 'motor', 'verbal', 'eye', 'wbc',
            'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw',
            'platelet_count', 'aptt_aptr', 'pt', 'alt', 'ast', 'serum_creatinine',
            'sodium', 'potassium', 'total_serum_bilirubin', 'lactate', 'pao2',
            'pao2_fio2', 'ph', 'high_senstivity_cardiac_troponin', 'esr',
            'inr', 'ferritin', 'd_dimer', 'crp', 'hs_crp', 'intubated']

# Dependent variable: Hospital Length of Stay (HLOS)
label = 'hospital_length_of_stay'

# Merge the 'Data-at-admission' and 'Hospital-length-of-stay' sheets based on patient ID
merged_data = pd.merge(data_at_admission, hospital_length_of_stay, on='id')

# Select only the relevant features and the target variable (HLOS)
selected_data = merged_data[features + [label]].copy()  # Ensure it's a copy to avoid SettingWithCopyWarning

selected_data.head()

,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,...,ph,high_senstivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,intubated,hospital_length_of_stay
0,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",99,62,85,20,93,...,NaN,NaN,NaN,1.1,NaN,NaN,223.0,NaN,No,21
1,61,Female,154.9,73.6,"""[\""Hypertension\"",\""Other\""]""",122,77,93,36,88,...,NaN,NaN,NaN,1.0,NaN,2354,95.6,NaN,No,5
2,58,Female,NaN,NaN,"""[\""Hypertension\""]""",109,72,73,24,91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,7
3,94,Male,182.8,66.2,"""[\""Hypertension\"",\""Other\""]""",165,71,92,16,96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,9
4,91,Male,NaN,NaN,"""[\""Chronic cardiac disease (not hypertension)...",131,76,60,18,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,9


In [6]:
non_numeric_columns = selected_data.select_dtypes(exclude=['number']).columns
print(non_numeric_columns)

Index(['sex', 'comorbidities', 'aptt_aptr', 'd_dimer', 'intubated'], dtype='object')


In [7]:
selected_data['d_dimer'] = selected_data['d_dimer'].replace({'<': '', '>': ''}, regex=True)

# Convert the cleaned column to numeric
selected_data['d_dimer'] = pd.to_numeric(selected_data['d_dimer'])

selected_data['aptt_aptr'] = selected_data['aptt_aptr'].replace({'<': '', '>': ''}, regex=True)

# Convert the cleaned column to numeric
selected_data['aptt_aptr'] = pd.to_numeric(selected_data['aptt_aptr'])

non_numeric_columns = selected_data.select_dtypes(exclude=['number']).columns
print(non_numeric_columns)

Index(['sex', 'comorbidities', 'intubated'], dtype='object')


In [8]:
numerical_columns = selected_data.select_dtypes(include=['number']).columns
selected_data[numerical_columns] = selected_data[numerical_columns].apply(lambda col: col.fillna(col.mean()))

# Drop rows with NaN in non-numerical columns
non_numerical_columns = selected_data.select_dtypes(exclude=['number']).columns
df = selected_data.dropna(subset=non_numerical_columns)

In [9]:
df.head()

,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,...,ph,high_senstivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,intubated,hospital_length_of_stay
0,74,Male,166.921591,80.232471,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",99,62,85,20,93,...,7.340333,NaN,61.8,1.100000,1011.526316,1320.822314,223.000000,NaN,No,21
1,61,Female,154.900000,73.600000,"""[\""Hypertension\"",\""Other\""]""",122,77,93,36,88,...,7.340333,NaN,61.8,1.000000,1011.526316,2354.000000,95.600000,NaN,No,5
2,58,Female,166.921591,80.232471,"""[\""Hypertension\""]""",109,72,73,24,91,...,7.340333,NaN,61.8,1.161513,1011.526316,1320.822314,109.707971,NaN,No,7
3,94,Male,182.800000,66.200000,"""[\""Hypertension\"",\""Other\""]""",165,71,92,16,96,...,7.340333,NaN,61.8,1.161513,1011.526316,1320.822314,109.707971,NaN,No,9
4,91,Male,166.921591,80.232471,"""[\""Chronic cardiac disease (not hypertension)...",131,76,60,18,95,...,7.340333,NaN,61.8,1.161513,1011.526316,1320.822314,109.707971,NaN,No,9


In [10]:
nan_columns = df.columns[df.isna().all()]
nan_columns

Index(['high_senstivity_cardiac_troponin', 'hs_crp'], dtype='object')

In [11]:
df = df.drop(columns=nan_columns)
df.head()

,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,...,pao2,pao2_fio2,ph,esr,inr,ferritin,d_dimer,crp,intubated,hospital_length_of_stay
0,74,Male,166.921591,80.232471,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",99,62,85,20,93,...,78.033333,1.5,7.340333,61.8,1.100000,1011.526316,1320.822314,223.000000,No,21
1,61,Female,154.900000,73.600000,"""[\""Hypertension\"",\""Other\""]""",122,77,93,36,88,...,78.033333,1.5,7.340333,61.8,1.000000,1011.526316,2354.000000,95.600000,No,5
2,58,Female,166.921591,80.232471,"""[\""Hypertension\""]""",109,72,73,24,91,...,78.033333,1.5,7.340333,61.8,1.161513,1011.526316,1320.822314,109.707971,No,7
3,94,Male,182.800000,66.200000,"""[\""Hypertension\"",\""Other\""]""",165,71,92,16,96,...,78.033333,1.5,7.340333,61.8,1.161513,1011.526316,1320.822314,109.707971,No,9
4,91,Male,166.921591,80.232471,"""[\""Chronic cardiac disease (not hypertension)...",131,76,60,18,95,...,78.033333,1.5,7.340333,61.8,1.161513,1011.526316,1320.822314,109.707971,No,9


In [12]:
label_encoder = LabelEncoder()

# Select non-numerical columns
non_numeric_columns = df.select_dtypes(exclude=['number']).columns

# Apply LabelEncoder on non-numerical columns
for col in non_numeric_columns:
    df[col] = label_encoder.fit_transform(df[col])

In [13]:
df.head()

,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,...,pao2,pao2_fio2,ph,esr,inr,ferritin,d_dimer,crp,intubated,hospital_length_of_stay
0,74,1,166.921591,80.232471,102,99,62,85,20,93,...,78.033333,1.5,7.340333,61.8,1.100000,1011.526316,1320.822314,223.000000,0,21
1,61,0,154.900000,73.600000,120,122,77,93,36,88,...,78.033333,1.5,7.340333,61.8,1.000000,1011.526316,2354.000000,95.600000,0,5
2,58,0,166.921591,80.232471,122,109,72,73,24,91,...,78.033333,1.5,7.340333,61.8,1.161513,1011.526316,1320.822314,109.707971,0,7
3,94,1,182.800000,66.200000,120,165,71,92,16,96,...,78.033333,1.5,7.340333,61.8,1.161513,1011.526316,1320.822314,109.707971,0,9
4,91,1,166.921591,80.232471,38,131,76,60,18,95,...,78.033333,1.5,7.340333,61.8,1.161513,1011.526316,1320.822314,109.707971,0,9


In [14]:
# Split the dataset into independent variables (X) and dependent variable (y)
features = ['age', 'sex', 'height', 'weight', 'comorbidities',
            'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate', 'respiratory_rate',
            'oxygen_saturation', 'temperature', 'motor', 'verbal', 'eye', 'wbc',
            'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw',
            'platelet_count', 'aptt_aptr', 'pt', 'alt', 'ast', 'serum_creatinine',
            'sodium', 'potassium', 'total_serum_bilirubin', 'lactate', 'pao2',
            'pao2_fio2', 'ph', 'esr',
            'inr', 'ferritin', 'd_dimer', 'crp', 'intubated']
X = df[features]
y = df[label]

# Split the dataset into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data (useful for models like MLP and RNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Report statistics: Number of rows and columns
num_rows, num_columns = selected_data.shape
print(f'Number of rows (patients): {num_rows}')
print(f'Number of columns (features from patients): {num_columns}')


Number of rows (patients): 494
Number of columns (features from patients): 44


In [15]:
#Statistics from the preprocessed data
#Report the following statistics from the data,ANSWERS included:

# Number of rows and columns in the preprocessed dataset
num_rows, num_columns = df.shape

# Compute the mean of each column
column_means = df.mean()

# Identify columns that have all blank (0) values
columns_with_all_zero_values = df.columns[(df == 0).all()]

# Intuitive guess for important features (based on correlation to health)
important_features = ['age', 'comorbidities', 'systolic_blood_pressure',
                      'diastolic_blood_pressure', 'heart_rate', 'oxygen_saturation',
                      'respiratory_rate', 'temperature', 'wbc', 'hemoglobin']

# Report the results
num_rows, num_columns, column_means, columns_with_all_zero_values, important_features


(494,
 42,
 age                           65.973684
 sex                            0.582996
 height                       166.921591
 weight                        80.232471
 comorbidities                 90.101215
 systolic_blood_pressure      129.378543
 diastolic_blood_pressure      75.410931
 heart_rate                    97.439271
 respiratory_rate              24.844130
 oxygen_saturation             93.153846
 temperature                   37.762348
 motor                          5.910569
 verbal                         4.768293
 eye                            3.943089
 wbc                            8.055691
 rbc                            4.630142
 hemoglobin                   129.876016
 hematocrit                     0.395108
 mcv                           85.935569
 mch                           28.256504
 mchc                         328.540650
 rdw                           13.975203
 platelet_count               231.967480
 aptt_aptr                     31.345324
 pt  

In [16]:
num_rows, num_columns = df.shape
print(f"Number of rows: {num_rows}, Number of columns: {num_columns}")

Number of rows: 494, Number of columns: 42


In [17]:
column_means = df.mean()
print(column_means)

age                           65.973684
sex                            0.582996
height                       166.921591
weight                        80.232471
comorbidities                 90.101215
systolic_blood_pressure      129.378543
diastolic_blood_pressure      75.410931
heart_rate                    97.439271
respiratory_rate              24.844130
oxygen_saturation             93.153846
temperature                   37.762348
motor                          5.910569
verbal                         4.768293
eye                            3.943089
wbc                            8.055691
rbc                            4.630142
hemoglobin                   129.876016
hematocrit                     0.395108
mcv                           85.935569
mch                           28.256504
mchc                         328.540650
rdw                           13.975203
platelet_count               231.967480
aptt_aptr                     31.345324
pt                            14.636184


In [18]:
columns_with_all_zero_values = df.columns[(df == 0).all()]
print(columns_with_all_zero_values)

Index([], dtype='object')


In [19]:
### MLP
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [20]:
def create_model(learning_rate=0.001, neurons=32, activation='relu'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(32, activation=activation))
    model.add(Dense(1))  # Output layer for regression
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [21]:
# Hyperparameter ranges to tune
batch_sizes = [16, 32, 64]
epochs = [50, 100]
learning_rates = [0.001, 0.01]
neuron_counts = [32, 64]
activations = ['relu', 'tanh']

# Variable to store the best model and its MSE
best_mse = float('inf')
best_params = None

In [22]:
for batch_size in batch_sizes:
    for epoch in epochs:
        for lr in learning_rates:
            for neurons in neuron_counts:
                for activation in activations:
                    print(f"Training with batch_size={batch_size}, epochs={epoch}, learning_rate={lr}, neurons={neurons}, activation={activation}")

                    # Create the model
                    model = create_model(learning_rate=lr, neurons=neurons, activation=activation)

                    # Train the model
                    model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, verbose=0)

                    # Evaluate the model
                    y_pred = model.predict(X_test)
                    mse = mean_squared_error(y_test, y_pred)

                    print(f"MSE: {mse}")

                    # Check if this model is better than the previous ones
                    if mse < best_mse:
                        best_mse = mse
                        best_params = {
                            'batch_size': batch_size,
                            'epochs': epoch,
                            'learning_rate': lr,
                            'neurons': neurons,
                            'activation': activation
                        }

# Output the best hyperparameters and MSE
print(f"Best hyperparameters: {best_params}")
print(f"Best MSE on test data: {best_mse}")

Training with batch_size=16, epochs=50, learning_rate=0.001, neurons=32, activation=relu


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
MSE: 107.51883444341182
Training with batch_size=16, epochs=50, learning_rate=0.001, neurons=32, activation=tanh


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
MSE: 81.83252125527235
Training with batch_size=16, epochs=50, learning_rate=0.001, neurons=64, activation=relu


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
MSE: 80.93656148765837
Training with batch_size=64, epochs=100, learning_rate=0.01, neurons=64, activation=relu


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
MSE: 127.33343761848545
Training with batch_size=64, epochs=100, learning_rate=0.01, neurons=64, activation=tanh


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
MSE: 78.88272784726371
Best hyperparameters: {'batch_size': 16, 'epochs': 50, 'learning_rate': 0.01, 'neurons': 64, 'activation': 'tanh'}
Best MSE on test data: 77.30638365316295


In [23]:
###RNN
# Load and inspect the "Days-breakdown" and "Hospital-length-of-stay" sheets
days_breakdown_df = pd.read_excel(xls, sheet_name='Days-breakdown')
hlos_df = pd.read_excel(xls, sheet_name='Hospital-length-of-stay')

# Display the first few rows of each dataframe to inspect the data
days_breakdown_df.head(), hlos_df.head()

(   id  parent_id  day  systolic_blood_pressure  diastolic_blood_pressure  \
 0   1          1    1                    119.0                      54.0   
 1   2          1    2                    133.0                      64.0   
 2   3          1    3                    140.0                      74.0   
 3   4          1    4                    154.0                      78.0   
 4   5          1    5                    155.0                      61.0   
 
    heart_rate  respiratory_rate  oxygen_saturation  temperature  \
 0        79.0              18.0               94.0         37.2   
 1        73.0              18.0               98.0         37.1   
 2        70.0              20.0               95.0         37.5   
 3        77.0              18.0               95.0         37.1   
 4        64.0              16.0               92.0         36.7   
 
    highest_mean_arterial_pressure  ...  wall_motion_abnormality  chest_x_ray  \
 0                            85.0  ...      

In [24]:
features = [
    'id', 'parent_id_x','day', 'systolic_blood_pressure', 'diastolic_blood_pressure',
    'heart_rate', 'respiratory_rate', 'oxygen_saturation', 'temperature',
    'highest_mean_arterial_pressure', 'lowest_mean_arterial_pressure', 'highest_heart_rate',
    'lowest_heart_rate', 'highest_creatinine', 'lowest_urine_output', 'highest_gcs',
    'lowest_gcs', 'intubated', 'highest_ph', 'lowest_ph', 'cardiac_arrest',
    'arrested_time', 'major_cardiac_events', 'clinically_diagnosed_infections',
    'mechanical_ventilation', 'number_of_vasoactive_agents', 'antiarrhythmic_therapies',
    'renal_replacement_therapy_dialysis', 'cardiovascular_mechanical_support',
    'fluid_balance', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'platelet_count',
    'aptt_aptr', 'pt', 'alt', 'ast', 'mch', 'mcv', 'mchc', 'rdw', 'serum_creatinine',
    'sodium', 'potassium', 'total_serum_bilirubin', 'lactate', 'pao2', 'pao2_fio2',
    'ph', 'high_sensitivity_cardiac_troponin', 'esr', 'inr', 'ferritin', 'd_dimer',
    'crp', 'hs_crp', 'echocardiogram', 'ejection_fraction', 'wall_motion_abnormality',
    'chest_x_ray', 'cxr_findings', 'chest_ct', 'chest_ct_findings', 'head_ct',
    'head_ct_findings', 'antimicrobial', 'anticoagulation', 'steroid'
]

# Dependent variable: Hospital Length of Stay (HLOS)
label = 'hospital_length_of_stay'

# Merge the 'Data-at-admission' and 'Hospital-length-of-stay' sheets based on patient ID
merged_data = pd.merge(days_breakdown, hospital_length_of_stay, on='id')
# Select only the relevant features and the target variable (HLOS)
selected_data = merged_data[features + [label]].copy()  # Ensure it's a copy to avoid SettingWithCopyWarning

selected_data.head()

,id,parent_id_x,day,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_mean_arterial_pressure,...,chest_x_ray,cxr_findings,chest_ct,chest_ct_findings,head_ct,head_ct_findings,antimicrobial,anticoagulation,steroid,hospital_length_of_stay
0,1,1,1,119.0,54.0,79.0,18.0,94.0,37.2,85.0,...,No,"""[\""Unilateral Consolidation\""]""",No,"""[\""Bilateral consolidation/infiltration\""]""",No,"""[]""",Yes,Yes,No,21
1,2,1,2,133.0,64.0,73.0,18.0,98.0,37.1,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,5
2,3,1,3,140.0,74.0,70.0,20.0,95.0,37.5,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,7
3,4,1,4,154.0,78.0,77.0,18.0,95.0,37.1,NaN,...,Yes,"""[\""Unilateral Consolidation\""]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,9
4,5,1,5,155.0,61.0,64.0,16.0,92.0,36.7,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,9


In [25]:
non_numeric_columns = selected_data.select_dtypes(exclude=['number']).columns
print(non_numeric_columns)

Index(['intubated', 'cardiac_arrest', 'major_cardiac_events',
       'clinically_diagnosed_infections', 'mechanical_ventilation',
       'antiarrhythmic_therapies', 'renal_replacement_therapy_dialysis',
       'cardiovascular_mechanical_support', 'd_dimer', 'echocardiogram',
       'wall_motion_abnormality', 'chest_x_ray', 'cxr_findings', 'chest_ct',
       'chest_ct_findings', 'head_ct', 'head_ct_findings', 'antimicrobial',
       'anticoagulation', 'steroid'],
      dtype='object')


In [26]:
selected_data['intubated'] = selected_data['intubated'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['intubated'] = pd.to_numeric(selected_data['intubated'])

selected_data['cardiac_arrest'] = selected_data['cardiac_arrest'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['cardiac_arrest'] = pd.to_numeric(selected_data['cardiac_arrest'])

selected_data['major_cardiac_events'] = selected_data['major_cardiac_events'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['major_cardiac_events'] = pd.to_numeric(selected_data['major_cardiac_events'])

selected_data['clinically_diagnosed_infections'] = selected_data['clinically_diagnosed_infections'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['clinically_diagnosed_infections'] = pd.to_numeric(selected_data['clinically_diagnosed_infections'])

selected_data['mechanical_ventilation'] = selected_data['mechanical_ventilation'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['mechanical_ventilation'] = pd.to_numeric(selected_data['mechanical_ventilation'])

selected_data['antiarrhythmic_therapies'] = selected_data['antiarrhythmic_therapies'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['antiarrhythmic_therapies'] = pd.to_numeric(selected_data['antiarrhythmic_therapies'])

selected_data['renal_replacement_therapy_dialysis'] = selected_data['renal_replacement_therapy_dialysis'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['renal_replacement_therapy_dialysis'] = pd.to_numeric(selected_data['renal_replacement_therapy_dialysis'])

selected_data['cardiovascular_mechanical_support'] = selected_data['cardiovascular_mechanical_support'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['cardiovascular_mechanical_support'] = pd.to_numeric(selected_data['cardiovascular_mechanical_support'])

selected_data['chest_ct'] = selected_data['chest_ct'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['chest_ct'] = pd.to_numeric(selected_data['chest_ct'])

selected_data['head_ct'] = selected_data['head_ct'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['head_ct'] = pd.to_numeric(selected_data['head_ct'])

selected_data['antimicrobial'] = selected_data['antimicrobial'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['antimicrobial'] = pd.to_numeric(selected_data['antimicrobial'])

selected_data['anticoagulation'] = selected_data['anticoagulation'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['anticoagulation'] = pd.to_numeric(selected_data['anticoagulation'])

selected_data['chest_x_ray'] = selected_data['chest_x_ray'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['chest_x_ray'] = pd.to_numeric(selected_data['chest_x_ray'])

selected_data['steroid'] = selected_data['steroid'].replace({'No': 0, 'Yes': 1}, regex=True)

# Convert the cleaned column to numeric
selected_data['steroid'] = pd.to_numeric(selected_data['steroid'])

non_numeric_columns = selected_data.select_dtypes(exclude=['number']).columns
print(non_numeric_columns)

Index(['d_dimer', 'echocardiogram', 'wall_motion_abnormality', 'cxr_findings',
       'chest_ct_findings', 'head_ct_findings'],
      dtype='object')


In [27]:
from sklearn.model_selection import train_test_split

# Split into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(selected_data, test_size=0.2, random_state=42)

# Ensure sorting by 'id' and 'day' for sequential modeling
train_data = train_data.sort_values(by=['id', 'day'])
test_data = test_data.sort_values(by=['id', 'day'])

# Update target label (HLOS) for each day
train_data['days_remaining'] = train_data.groupby('id')[label].transform(lambda x: x - (x.rank() - 1))
test_data['days_remaining'] = test_data.groupby('id')[label].transform(lambda x: x - (x.rank() - 1))

# Drop original HLOS label since we have the transformed 'days_remaining' label
train_data = train_data.drop(columns=[label])
test_data = test_data.drop(columns=[label])

# Check the head of the training data
train_data.head()

,id,parent_id_x,day,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_mean_arterial_pressure,...,chest_x_ray,cxr_findings,chest_ct,chest_ct_findings,head_ct,head_ct_findings,antimicrobial,anticoagulation,steroid,days_remaining
1,2,1,2,133.0,64.0,73.0,18.0,98.0,37.1,NaN,...,0,"""[]""",0,"""[]""",0,"""[]""",1,1,1,5.0
3,4,1,4,154.0,78.0,77.0,18.0,95.0,37.1,NaN,...,1,"""[\""Unilateral Consolidation\""]""",0,"""[]""",0,"""[]""",1,1,1,9.0
4,5,1,5,155.0,61.0,64.0,16.0,92.0,36.7,NaN,...,0,"""[]""",0,"""[]""",0,"""[]""",1,1,1,9.0
5,6,1,6,156.0,74.0,62.0,18.0,92.0,36.9,NaN,...,0,"""[]""",0,"""[]""",0,"""[]""",1,1,1,32.0
6,7,1,7,158.0,69.0,81.0,18.0,92.0,36.5,NaN,...,0,"""[]""",0,"""[]""",0,"""[]""",1,1,1,4.0


In [28]:
merged_df = pd.merge(days_breakdown_df, hlos_df[['parent_id', 'hospital_length_of_stay']],
                     on='parent_id', how='inner')

# Create the target variable 'remaining_days' = HLOS - day
merged_df['remaining_days'] = merged_df['hospital_length_of_stay'] - merged_df['day']

# Display the first few rows of the merged dataframe to verify the restructuring
merged_df.head()

,id,parent_id,day,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_mean_arterial_pressure,...,cxr_findings,chest_ct,chest_ct_findings,head_ct,head_ct_findings,antimicrobial,anticoagulation,steroid,hospital_length_of_stay,remaining_days
0,1,1,1,119.0,54.0,79.0,18.0,94.0,37.2,85.0,...,"""[\""Unilateral Consolidation\""]""",No,"""[\""Bilateral consolidation/infiltration\""]""",No,"""[]""",Yes,Yes,No,21,20
1,2,1,2,133.0,64.0,73.0,18.0,98.0,37.1,NaN,...,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,21,19
2,3,1,3,140.0,74.0,70.0,20.0,95.0,37.5,NaN,...,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,21,18
3,4,1,4,154.0,78.0,77.0,18.0,95.0,37.1,NaN,...,"""[\""Unilateral Consolidation\""]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,21,17
4,5,1,5,155.0,61.0,64.0,16.0,92.0,36.7,NaN,...,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,21,16


In [29]:
###Statistics from pre processed data
# Number of rows and columns in the preprocessed dataset
num_rows, num_columns = df.shape

# Compute the mean of each column
column_means = df.mean()

# Identify columns that have all blank (0) values
columns_with_all_zero_values = df.columns[(df == 0).all()]

# Intuitive guess for important features (based on correlation to health)
important_features = ['age', 'comorbidities', 'systolic_blood_pressure',
                      'diastolic_blood_pressure', 'heart_rate', 'oxygen_saturation',
                      'respiratory_rate', 'temperature', 'wbc', 'hemoglobin']

# Report the results
num_rows, num_columns, column_means, columns_with_all_zero_values, important_features

(494,
 42,
 age                           65.973684
 sex                            0.582996
 height                       166.921591
 weight                        80.232471
 comorbidities                 90.101215
 systolic_blood_pressure      129.378543
 diastolic_blood_pressure      75.410931
 heart_rate                    97.439271
 respiratory_rate              24.844130
 oxygen_saturation             93.153846
 temperature                   37.762348
 motor                          5.910569
 verbal                         4.768293
 eye                            3.943089
 wbc                            8.055691
 rbc                            4.630142
 hemoglobin                   129.876016
 hematocrit                     0.395108
 mcv                           85.935569
 mch                           28.256504
 mchc                         328.540650
 rdw                           13.975203
 platelet_count               231.967480
 aptt_aptr                     31.345324
 pt  

In [30]:
num_rows, num_columns = df.shape
print(f"Number of rows: {num_rows}, Number of columns: {num_columns}")

Number of rows: 494, Number of columns: 42


In [31]:
column_means = df.mean()
print(column_means)

age                           65.973684
sex                            0.582996
height                       166.921591
weight                        80.232471
comorbidities                 90.101215
systolic_blood_pressure      129.378543
diastolic_blood_pressure      75.410931
heart_rate                    97.439271
respiratory_rate              24.844130
oxygen_saturation             93.153846
temperature                   37.762348
motor                          5.910569
verbal                         4.768293
eye                            3.943089
wbc                            8.055691
rbc                            4.630142
hemoglobin                   129.876016
hematocrit                     0.395108
mcv                           85.935569
mch                           28.256504
mchc                         328.540650
rdw                           13.975203
platelet_count               231.967480
aptt_aptr                     31.345324
pt                            14.636184


In [32]:
columns_with_all_zero_values = df.columns[(df == 0).all()]
print(columns_with_all_zero_values)

Index([], dtype='object')


In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Features and target variable
features = ['systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate',
            'respiratory_rate', 'oxygen_saturation', 'temperature']

# Drop rows with missing data in the selected features or target variable
merged_df = merged_df.dropna(subset=features + ['hospital_length_of_stay'])

# Create 'remaining_days' as H-1, H-2, etc., for each patient
merged_df['remaining_days'] = merged_df.groupby('id')['hospital_length_of_stay'].transform(lambda x: x - np.arange(len(x)))

# Extract feature matrix (X) and target vector (y)
X = merged_df[features].values
y = merged_df['remaining_days'].values

# Reshape data for sequential modeling
def reshape_for_rnn(X, y, time_steps):
    X_reshaped, y_reshaped = [], []
    for i in range(len(X) - time_steps + 1):
        X_reshaped.append(X[i:i + time_steps])
        y_reshaped.append(y[i + time_steps - 1])  # Target for the last time step
    return np.array(X_reshaped), np.array(y_reshaped)

# Set the number of time steps (days per patient)
time_steps = 7
X_seq, y_seq = reshape_for_rnn(X, y, time_steps)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Hyperparameter options
batch_sizes = [16, 32, 64]
epochs_list = [10, 20, 50]
learning_rates = [0.001, 0.0001]  # Learning rates for optimizers
neuron_counts = [20, 50, 100]
activations = ['relu', 'tanh']

# To store the best results
best_mse = float('inf')
best_params = {}

# Iterate over the hyperparameters to tune the model
for batch_size in batch_sizes:
    for epoch in epochs_list:
        for lr in learning_rates:
            for neurons in neuron_counts:
                for activation in activations:
                    print(f"Training with batch_size={batch_size}, epochs={epoch}, learning_rate={lr}, neurons={neurons}, activation={activation}")

                    # Build the RNN model
                    model = Sequential()
                    model.add(SimpleRNN(neurons, input_shape=(X_train.shape[1], X_train.shape[2]), activation=activation))
                    model.add(Dense(1))  # Output layer for regression

                    # Compile the model with learning rate
                    optimizer = Adam(learning_rate=lr)
                    model.compile(optimizer=optimizer, loss='mean_squared_error')

                    # Train the model
                    model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, verbose=0)

                    # Evaluate the model
                    y_pred = model.predict(X_test)
                    mse = mean_squared_error(y_test, y_pred)

                    print(f"MSE: {mse}")

                    # Check if this model is better than the previous ones
                    if mse < best_mse:
                        best_mse = mse
                        best_params = {
                            'batch_size': batch_size,
                            'epochs': epoch,
                            'learning_rate': lr,
                            'neurons': neurons,
                            'activation': activation
                        }

# Output the best hyperparameters and MSE
print(f"Best hyperparameters: {best_params}")
print(f"Best MSE on test data: {best_mse}")

Training with batch_size=16, epochs=10, learning_rate=0.001, neurons=20, activation=relu


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
MSE: 166.80158370533206
Training with batch_size=16, epochs=10, learning_rate=0.001, neurons=20, activation=tanh


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
MSE: 162.62526761942954
Training with batch_size=16, epochs=10, learning_rate=0.001, neurons=50, activation=relu


...

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
MSE: 179.7591615751008
Training with batch_size=64, epochs=50, learning_rate=0.0001, neurons=100, activation=relu


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
MSE: 152.6695468334816
Training with batch_size=64, epochs=50, learning_rate=0.0001, neurons=100, activation=tanh


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
MSE: 162.57538657668331
Best hyperparameters: {'batch_size': 64, 'epochs': 50, 'learning_rate': 0.0001, 'neurons': 100, 'activation': 'relu'}
Best MSE on test data: 152.6695468334816
